# Sending and image to the prediction service. 

We will show 2 ways that you can use to send the image to the prediction service you have just deployed. 

As soon as you provide the parameters we need:

## Parameters

You need to provide 2 parameters: 

* the url of your service
* the name (path) of the image to send to it

In [ ]:
## we need a couple things:
!pip install ipyfilechooser --quiet
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import requests

### The URL of the service

In [ ]:
## this function will validate the URL entered in the box
def validate(url):
    print("You provided the following URL:" + url )
    
    ## make sure they changed the URL from the default value. 
    assert "DUMMYURL" not in url , "YOU DID NOT UPDATE THE DEFAULT URL!"
    
    ## make sure the url is a website
    try:
        r = requests.head(url)
        print("HTTP Code returned: " + str(r.status_code))
        # prints the int of the status code. Find more at httpstatusrappers.com :)
        assert r.status_code == 200 , "You sure about that? That URL is not returning an HTTP code 200. "
        print ("Seems good from what I can tell  " + u'\u2713')

        ## at this point, we are ready to define the url as being valid:
        global valid_url
        valid_url = url

    except requests.ConnectionError:
        print("failed to connect")

## change the default name of the button
my_interact_manual = interact_manual.options(manual_name="Validate that URL")

print("Enter below the URL of your deployed service")

## display the box and the button, with a default value
my_url = my_interact_manual(validate, url=widgets.Text(value='http://DUMMYURL.apps.pilot.j61u.p1.openshiftapps.com/',description='URL:',layout=widgets.Layout(width='80%')));

In [ ]:
## make sure the valid_url got created
assert valid_url , "Something is off"

## not that we don't trust you, but let's verify actually updated the above text box
assert "DUMMYURL" not in valid_url , "YOU DID NOT UPDATE THE DEFAULT URL!"

## no output is a good sign

In [ ]:
## and now let's make sure it's at least a functional website. 
import requests
try:
    r = requests.head(valid_url)
    print("HTTP Code returned: " + str(r.status_code))
    # prints the int of the status code. Find more at httpstatusrappers.com :)
    assert r.status_code == 200 , "You sure about that? That URL is not returning an HTTP code 200. "
except requests.ConnectionError:
    print("failed to connect")

## no output is a good sign

### The Image to upload

In [ ]:
from ipyfilechooser import FileChooser
fc = FileChooser('./')

fc.filter_pattern = ['*.jpg', '*.png', '*.jpeg']

# Change the title (use '' to hide)
fc.title = '<b>Choose an image file</b>'
display(fc)

In [ ]:
assert fc.selected , "You need to choose a picture!"

print("The chosen file is:" + fc.selected)

### Review the parameters and their values

In [ ]:
my_image = fc.selected
my_route = valid_url

print("URL of prediction server: " + my_route)
print("image path: " + my_image)
file = open(fc.selected, "rb")
image = file.read()
widgets.Image(
    value=image,
    format='png',
    width=300,
    height=400,
)

## Submitting an image to the service using python

In [ ]:
import base64
import requests
from json import dumps

with open(my_image, "rb") as image_file:
    encoded_image = base64.b64encode(image_file.read()).decode('utf-8')
content = {"image": encoded_image}
json_data = dumps(content)

headers = {"Content-Type" : "application/json"}

r = requests.post(my_route + '/predictions', data=json_data, headers=headers)

print(r.content)

## Submitting and image to the service using `curl`

In [ ]:
# this format of command does not work right now, as far as I can tell. 
#CURL_COMMAND1 = 'curl -X POST -H "Content-Type: application/json" --data ' + "'{\"data\": \"" + my_image + "\"}' " + my_route + "predictions"
# print (CURL_COMMAND)

CURL_COMMAND2 = "( echo -n '{\"image\": \"'; base64 " + my_image + "; echo '\"}' ) | curl -H \"Content-Type: application/json\" -d @- " + my_route + "/predictions "

print ("this is the command to execute at the prompt:\n\n" + "     " + CURL_COMMAND2 + '\n')

print ("And this is the result" + '\n')
!{CURL_COMMAND2}